In [23]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 
from collections import deque

import qubx
%qubxd

%load_ext autoreload
%autoreload 2

import tests.qubx.ta.utils_for_testing as test
from qubx.utils import fig, sbp

from qubx.data.readers import (
    CsvDataReader, QuotesDataProcessor, OhlcvDataProcessor, QuotesFromOHLCVDataProcessor, 
    OhlcvPandasDataProcessor, STOCK_DAILY_SESSION
)
from qubx.core.series import TimeSeries, OHLCV
from qubx.ta.indicators import (sma, ema, tema, dema, kama, highest, lowest)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## <font color='gray'>Reload dev environment</font>

In [ ]:
from qubx.core.series import TimeSeries; from qubx.ta.indicators import sma; del sys.modules['qubx.core.series']; del sys.modules['qubx.ta.indicators']
from qubx.core.series import TimeSeries, OHLCV
from qubx.core.utils import recognize_time
from qubx.ta.indicators import (sma, ema, tema, dema, kama, highest, lowest)

## <font color='red'><b>On formed bar experiments</b></font>

In [55]:
# r2x = CsvDataReader('../tests/data/csv/BTCUSDT_ohlcv_M1_sec.csv.gz', QuotesFromOHLCVDataProcessor(trades=1))
# ticks = r2x.read()
# - small storage
r0 = CsvDataReader('../tests/data/csv/quotes.csv', QuotesDataProcessor())
ticks = r0.read()
len(ticks)

500

In [69]:
s0 = TimeSeries('T0', '1Min')
control = TimeSeries('T0', '1Min')
m0 = sma(s0, 3)
for q in ticks:
    s0.update(q.time, 0.5*(q.ask + q.bid))
    control.update(q.time, 0.5*(q.ask + q.bid))

m1 = sma(control, 3)
mx = test.scols(s0, m0, m1, names=['series', 'streamed', 'finished']).dropna()
assert test.N(mx.streamed) == mx.finished
mx

,series,streamed,finished
2017-08-24 13:04:00,9.815,9.821667,9.821667
2017-08-24 13:05:00,9.915,9.805000,9.805000
2017-08-24 13:06:00,9.835,9.855000,9.855000
2017-08-24 13:07:00,9.530,9.760000,9.760000
2017-08-24 13:08:00,9.425,9.596667,9.596667
2017-08-24 13:09:00,9.375,9.443333,9.443333
